# Part 2: Indexing and Evaluation

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Indexing

### 1.1 Build inverted index

After having pre-processed the data, you can then create the inverted index.

HINT - you may use the vocabulary data structure, like the one seen during the
Practical Labs:

{
    Term_id_1: [document_1, document_2, document_4],
    Term_id_2: [document_1, document_3, document_5, document_6],
    etc…
}

Important: For this assignment, we will be using conjunctive queries (AND). This means that every returned document must contain all the words from the query in order to be considered a match.

In [ ]:
# TODO: ...

### 1.2 Propose test queries

Define five queries that will be used to evaluate your search engine. (Be creative 😉)

HINT: How to choose the queries? The selection of the queries is up to you, but it’s suggested to select terms based on the popularity (keywords ranked by term frequencies or by TF-IDF, etc).

In [ ]:
# TODO: ...

### 1.3 Rank your results

Implement the TF-IDF algorithm and provide ranking-based results.

In [ ]:
# TODO: ...

## 2. Evaluation

### 2.1 Implement the following evaluation metrics to assess the effectiveness of your retrieval solutions. 

These metrics will help you measure how well your system retrieves relevant documents for each query:

i. Precision@K (P@K)

ii. Recall@K (R@K)

iii. Average Precision@K (P@K)

iv. F1-Score@K

v. Mean Average Precision (MAP)

vi. Mean Reciprocal Rank (MRR)

vii. Normalized Discounted Cumulative Gain (NDCG)

In [ ]:
# TODO: ...

### 2.2 Apply Evaluation Metrics
Apply the evaluation metrics you have implemented to the search results and relevance judgments provided in validation_labels.csv for the predefined queries. When reporting evaluation results, provide only numeric values, rounded to three decimal places. Do not include textual explanations or additional statistics in this section.

a. Query 1: women full sleeve sweatshirt cotton

b. Query 2: men slim jeans blue

In [ ]:
# TODO: ...

### 2.3 Relevance Judgments and Analysis

You will act as expert judges by establishing the ground truth for each document and query.

a. For the test queries you defined in Part 1, Step 2 during indexing, assign a binary relevance label to each document: 1 if the document is relevant to the query, or 0 if it is not.

b. Comment on each of the evaluation metrics, stating how they differ, and which information gives each of them. Analyze your results.

c. Analyze the current search system and identify its main problems or limitations. For each issue you find, propose possible ways to resolve it. Consider aspects such as retrieval accuracy, ranking quality, handling of different field types, query formulation, and indexing strategies.

In [ ]:
# TODO: ...